# Clique Detection Using the `NETWORK` Actionset in SAS Viya and Python

In this demonstration, we load a small representative graph, use our CAS server to calculate the maximal cliques in a graph.

Recall that a clique of graph G is an induced subgraph that is a complete graph. Every node in a clique is connected to every other node in that clique. A maximal clique is a clique that is not a subset of the nodes of any larger clique. That is, it is a set C of nodes such that every pair of nodes in C is connected by a link and every node not in C is missing a link to at least one node in C.

In the graph G below, we expect to find *four* cliques.

----------------

The basic flow of this notebook is as follows:
1. Load the sample graph into a Pandas DataFrame as a set of links that represent the total graph. 
2. Connect to our CAS server and load the actionsets we require.
3. Upload our sample graph to our CAS server.
4. Execute the clique detection and output results into CAS tables.
5. Do required data manipulation to render the network graphs that show each clique.
6. Prepare and display the network plots showing the cliques.

----------------
__Prepared by:__
Damian Herrick (damian.herrick@sas.com)

#### Imports

Our imports are broken out as follows:

| Module        | Description                                                                        |
|:--------------|:----------------------------------------------------------------------------------:|
| `os`          | Allows access to environment variables.                                            |
| `swat`        | SAS Python module that orchestrates communicatoin with a CAS server.               |
| `pandas`      | Data management module we use for preparation of local data.                       |
| `networkx`    | Used to manage graph data structures when plotting.                                |
| `bokeh`       | Module used to generate interactive plots of graphs.                               |

In [1]:
import os
from typing import Any, List

import swat
import pandas as pd

import networkx as nx
from networkx import Graph

from bokeh.core.enums import Palette
from bokeh.io import output_notebook, show, save
from bokeh.layouts import gridplot
from bokeh.models import Circle, MultiLine, Range1d
from bokeh.models.annotations import LabelSet
from bokeh.models.graphs import NodesAndLinkedEdges
from bokeh.models.plots import Plot
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Blues8
from bokeh.plotting import from_networkx

NX_SPRING_SEED = 96201546

In [2]:
def set_node_colors(
    graph: Graph, attr_to_highlight: Any, ref_attr: str, color_palette: Palette
) -> None:
    """set_node_colors: set colors based on binary settings.

    Args:
        graph (Graph): The input graph.
        attr_to_highlight (Any): Value of the attribute to highlight.
        ref_attr (str): label of the attribute.
        color_palette (Palette): Bokeh palette to use.
    """
    for node in graph.nodes():
        graph.nodes[node]["highlight"] = color_palette[graph.nodes[node]["clique"]]

def render_plot(
    graph: Graph,
    title: str,
    hover_tooltips: List,
    node_size: int = 15,
    node_color: str = Blues8[-1],
    node_alpha: float = 1.0,
    aspect_ratio: float = None,
    width: int = 600,
    height: int = 600,
    outfile: str = None,
) -> Plot:
    """render_plot :: simple function to plot a graph using bokeh.

    Args:
        graph (Graph): The input, fully prepared graph.
        title (str): Title of the graph we're creating.
        hover_tooltips (List): the list of tuples that should display on hover.
        node_size (int): Optional, default 15. Set the size of the nodes.
        node_color (str): Optional. attribute label with each node's color.
        node_alpha (float): Optional. Background color transparency.
        aspect_ratio (float): Optional. Defined aspect ratio of the figure.
        width (int): Optional, default 600px. Non-negative integer pixel width.
        height (int): Optional, default 600px. Non-negative integer pixel height.
        outfile (str): Optional. Name of the export file. Defaults to None.

    Returns:
        Plot: The prepared Bokeh figure ready for display.
    """
    # Create a plot — set dimensions, toolbar, and title
    plot = Plot(
        x_range=Range1d(-10.1, 10.1),
        title=title,
        width=width,
        height=height,
        aspect_ratio=aspect_ratio,
    )

    plot.xgrid.grid_line_color = None
    plot.ygrid.grid_line_color = None

    # Create a network graph object with spring layout
    nw_graph = from_networkx(
        graph, nx.spring_layout, scale=10, center=(2, 0), seed=NX_SPRING_SEED
    )

    # Set node size and color
    nw_graph.node_renderer.glyph = Circle(
        size=node_size, fill_color=node_color, fill_alpha=node_alpha
    )

    # Set edge opacity and width
    nw_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=2)

    # green hover for both nodes and edges
    nw_graph.node_renderer.hover_glyph = Circle(size=node_size, fill_color="#abdda4")
    nw_graph.edge_renderer.hover_glyph = MultiLine(line_color="#abdda4", line_width=4)

    nw_graph.inspection_policy = NodesAndLinkedEdges()

    plot.add_tools(HoverTool(tooltips=hover_tooltips))

    # Add network graph to the plot
    plot.renderers.append(nw_graph)

    # Add Labels
    x, y = zip(*nw_graph.layout_provider.graph_layout.values())
    node_labels = list(graph.nodes())
    source = ColumnDataSource(
        {"x": x, "y": y, "name": [node_labels[i] for i in range(len(x))]}
    )
    labels = LabelSet(
        x="x",
        x_offset=-5,
        y="y",
        y_offset=-5,
        text="name",
        source=source,
        background_fill_color=node_color,
        text_font_size="13px",
        background_fill_alpha=node_alpha,
    )
    plot.renderers.append(labels)

    if outfile is not None:
        save(plot, filename=outfile)

    return plot

The call to `output_notebook` is required by `bokeh` to render plots inside Jupyter Notebooks.

In [3]:
output_notebook()

Loading BokehJS ...

### Step 1: Get the edge list for the sample graph.

In [4]:
colNames = ["from", "to"]
links = [
    (0, 1),
    (0, 2),
    (0, 3),
    (0, 4),
    (0, 5),
    (0, 6),
    (1, 2),
    (1, 3),
    (1, 4),
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (3, 4),
    (5, 6),
    (7, 8),
    (8, 9),
]

dfLinkSetIn = pd.DataFrame(links, columns=colNames)

In [5]:
G = nx.from_pandas_edgelist(dfLinkSetIn, 'from', 'to')

Let's see what the graph looks like before we start looking for cliques.

In [6]:
# Choose a title
title = 'Sample Graph Used for Maximal Clique Detection'

# Define the label for the nodes.
hover = [("Node", "@index")]

# How big are the nodes?
nodeSize = 25

# send to the utility function to generate the plot.
plot = render_plot(G, title, hover, nodeSize)

# display the graph
show(plot)

### Step 2: Connect to CAS, load the actionsets we'll need, and upload our graph to the CAS server.

In [7]:
host = os.environ['CAS_HOST_ORGRD']
port = int(os.environ['CAS_PORT'])
print(f"{host}:{port}")

MissingSectionHeaderError: File contains no section headers.
file: '../../../common/conf/environment.txt', line: 1
'CAS_SERVER_HOST=orgrd061.unx.sas.com\n'

In [ ]:
# Connect to the server
conn = swat.CAS(host, port)

# Load the actionsets we need
_ = conn.loadactionset('network')
_ = conn.loadactionset('fedsql')

Before we load the data, we should verify which caslib is active. Since we just connected and have not specified, the active library should map to our user ID.

Only one caslib can be active at a time. As long as we are happy with the active caslib, we do not need to reference the caslib in subsequent calls to CAS through `swat` methods. Note that this is slightly different from the corrresponding CASL calls we reference.

In [ ]:
conn.getsessopt('caslib')

In [ ]:
# Upload the local data into CAS.
_ = conn.upload(dfLinkSetIn, casout=dict(name='LinkSetIn'))

### Step 3: Calculate the maximal cliques in our graph using the `network` actionset.

Since we've loaded our actionset, we can reference it using dot notation from our `connection` object.

Note that the Python code below is equivalent to this block of CASL:
```
proc network
   links         = mycas.LinkSetIn
   outNodes      = mycas.NodeSetOut;
   clique
      out        = mycas.Cliques
      maxCliques = all;
run;
```

In [ ]:
conn.network.clique(links=dict(name='LinkSetIn'),
                    outnodes=dict(name='nodeSetOut'), 
                    out=dict(name='cliques',), 
                    maxcliques='ALL')

### Step 4: Gather the results from CAS and prepare the data for plotting.

Initially, we want to get a summary count of the cliques. Since the `clique` action doesn't provide a summary of total cliques, and nodes associated with each, so we use a simple `fedsql` groupby call to generate that table.

In [ ]:
conn.fedsql.execdirect(query="create table cliqueSizes as select clique, count(*) from cliques group by clique")

If we want to see the contents of that table, we simply use the `fetch` action provided by `swat` to pull data locally and display in our notebook.

The output table confirms that the `network.clique` action found four cliques in our graph.

In [ ]:
conn.fetch(table=dict(name='cliqueSizes'), 
           sortby=[{'name':'clique', 
                    'order':'ascending'}
                  ]
          )

CAS has calculated all of the information we need, so let's get anything remote back down to our local machine.

Once we have it all we can prepare our plots.

Our end product is a 2x2 grid of network plots, each one highlighting a particular clique. We have the following tables in CAS:

| Table | Description |
|-------|-------------|
| `LinkSetIn` | The initial link set that defines our graph. |
| `NodeSetOut` | A simple table of nodes. |
| `cliques` | A results table that highlights which nodes belong to which cliques. Note this only shows nodes the __belong__ to a clique; nodes outside the clique are not shown |
| `cliqueSizes` | The summary table that shows the count of nodes in a particular clique. Useful to get a list of total cliques. |

For this exercise, we will use the `networkx` `Graph` that we created earlier. Thus we do not need the `LinkSetIn` or `NodeSetOut` tables.

We need to fetch the `cliques` and `cliqueSizes` tables from CAS, and we'll manipulate them so we can add both clique membership and preferred coloring as node attributes to our graph.

We fetch the `nodeSetOut` table and convert it to a list of nodes for simplicity.

All of this will be used to create mapping dictionaries with node as a key, and the value represents an attribute. Once the mapping dictionaries are ready they are easily added to our `Graph` with the `set_node_attributes` method.

----------------------------
`clique_count` : We need the clique count list later when we iterate over cliques to highlight each clique in a separate plot. Let's generate this list by making a `CASTable` object, that we then convert to a dictionary. That dictionary has two keys: `clique` and `COUNT`; our list is the keys in the `clique` dictionary.

In [ ]:
clique_count = list(conn.CASTable('cliqueSizes').to_dict()['clique'].keys())

`clique_list`: Now we do the same for the clique list, but we orient the resulting dictionary as `records` per Pandas. This gives us a JSON-like list of dictionaries. This list will then be used later to create our mapping dictionaries.

In [ ]:
clique_list = conn.CASTable('cliques').to_dict(orient='records')

### Step 5: Create base maps used to guide the individual plot generation.

We'll leverage a few Python conveniences over the next few cells - particularly comprehensions and easy list creation.

The results are:
`clique_map`: a dictionary keyed by node label, with all `False` values.
`highlight_map`: a dictionary keyed by node label, with the lightest color in the `Blues8` palette.

When we are creating the attributes for a given clique, member nodes will be assigned `True` in the `clique_map`, and a darker color from the `Blues8` palette.

In [ ]:
node_list = [int(x) for x in list(conn.CASTable('nodeSetOut')['node'])]
base_clique = [False]*len(node_list)
base_highlight = [Blues8[-1]]*len(node_list)

clique_map = dict(zip(node_list, base_clique))
highlight_map = dict(zip(node_list, base_highlight))

### Step 6: Assemble and Display the Plots

We iterate over clique count, assigning attributes to the network that guide hover labeling and node shading.

In [ ]:
# node size should be common over all plots.
nodeSize = 40

# initialize our plot container.
plot_list = []

# iterate over each of the cliques, and highlight (darken) the clique
for i in clique_count:
    # create a list of only the nodes that are members of the clique.
    subnode_list = [item['node'] for item in clique_list if item['clique'] == i]
    # create a clique and highlight map
    this_clique_map = {k:True if k in subnode_list else v for k,v in clique_map.items()}
    this_highlight_map = {k:Blues8[-4] if k in subnode_list else v for k,v in highlight_map.items()}
    # set our node attributes.
    nx.set_node_attributes(G, this_highlight_map, 'highlight')
    nx.set_node_attributes(G, this_clique_map, 'clique')
    # set the title and hover plot values.
    title = f"Clique {i}"
    hover = [("Node", "@index"), ("Clique", "@clique")]
    # render the plot and add it to the list
    plot_list.append(render_plot(G, title, hover, node_size=nodeSize, node_color='highlight'))

# now we know the length of the list, define how many columns we want, make our grid, and finally show it.
columns = int(len(plot_list) / 2)
grid = gridplot(plot_list, ncols=columns)
show(grid)

### Step 7: Clean up everything. 

Make sure we know what tables we created, drop them, and close our connection.
(This is probably overkill, since everything in this session is ephemeral anyway, but good practice nonetheless.

In [ ]:
conn.tableinfo()

In [ ]:
conn.droptable(name='LinkSetIn', quiet=True)
conn.droptable(name='nodeSetOut', quiet=True)
conn.droptable(name='cliques', quiet=True)
conn.droptable(name='cliqueSizes', quiet=True)

In [ ]:
conn.close()